# **Train Deep Reinforcement Learning Agent**




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Install and Import Required Libraries

In [2]:
!apt install swig cmake

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 0s (8,354 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 123595 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

In [3]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

INFO: pip is looking at multiple versions of gymnasium[box2d] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinu

During the notebook, we'll need to generate a replay video. To do so, with colab, **we need to have a virtual screen to be able to render the environment** (and thus record the frames).

Hence the following cell will install virtual screen libraries and create and run a virtual screen 🖥

In [4]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,135 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,221 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,944 kB]
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3

To make sure the new installed libraries are used, **sometimes it's required to restart the notebook runtime**. The next cell will force the **runtime to crash, so you'll need to connect again and run the code starting from here**. Thanks to this trick, **we will be able to run our virtual screen.**

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [8]:
import gymnasium as gym

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import notebook_login # To log to our Hugging Face account to be able to upload models to the Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 2. Create Vectorized Environment

- We create a vectorized environment (a method for stacking multiple independent environments into a single environment) of 16 environments, this way, **we'll have more diverse experiences during the training.**

In [3]:
# Create the environment
env = make_vec_env('LunarLander-v2', n_envs=16)

## 3. Create the Model

In [4]:
# We use MultiLayerPerceptron (MLPPolicy) because the input is a vector,
# if we had frames as input we would use CnnPolicy
# model = PPO('MlpPolicy', env, verbose=1)
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1)

Using cuda device


## 4. Train the Agent 🏃
- Let's train our agent for 1,000,000 timesteps, don't forget to use GPU on Colab. It will take approximately ~20min, but you can use fewer timesteps if you just want to try it out.
- During the training, take a ☕ break you deserved it 🤗

In [5]:
# Train it for 1,000,000 timesteps
model.learn(total_timesteps=1000000, progress_bar=True)

# Specify file name for model and save the model to file
model_name = "ppo-LunarLander-v2"
model.save(model_name)

Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 91.4     |
|    ep_rew_mean     | -185     |
| time/              |          |
|    fps             | 1812     |
|    iterations      | 1        |
|    time_elapsed    | 9        |
|    total_timesteps | 16384    |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 90.9       |
|    ep_rew_mean          | -150       |
| time/                   |            |
|    fps                  | 1612       |
|    iterations           | 2          |
|    time_elapsed         | 20         |
|    total_timesteps      | 32768      |
| train/                  |            |
|    approx_kl            | 0.00700642 |
|    clip_fraction        | 0.023      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.38      |
|    explained_variance   | -0.00286   |
|    learning_rate        | 0.0003     |
|   

## 5. Evaluate the agent

In [9]:
# Evaluate the agent
# Create a new environment for evaluation
eval_env = Monitor(gym.make("LunarLander-v2", render_mode='rgb_array'))

# Evaluate the model with 10 evaluation episodes and deterministic=True
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)

# Print the results
print(f"mean_reward = {mean_reward:.2f} +/- {std_reward:.2f} (standard deviation reward)")

mean_reward = 258.18 +/- 20.00 (standard deviation reward)


## 6. Upload to Repo at HF

In [ ]:
# # Login with token into HF repo
# notebook_login()
# !git config --global credential.helper store

In [ ]:
# # Upload the model onto Hugging Face repo and generate the video
# import gymnasium as gym
# from stable_baselines3.common.vec_env import DummyVecEnv
# from stable_baselines3.common.env_util import make_vec_env

# from huggingface_sb3 import package_to_hub

# ## TODO: Define a repo_id
# ## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
# repo_id = "vagi/ppo-LunarLander-v2"

# # TODO: Define the name of the environment
# env_id = "LunarLander-v2"

# # Create the evaluation env and set the render_mode="rgb_array"
# eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode="rgb_array"))])


# # TODO: Define the model architecture we used
# model_architecture = "PPO"

# ## TODO: Define the commit message
# commit_message = "PPO LunarLander-v2 trained agent - version 01"

# # method save, evaluate, generate a model card and record a replay video of your agent before pushing the repo to the hub
# package_to_hub(model=model, # Our trained model
#                model_name=model_name, # The name of our trained model
#                model_architecture=model_architecture, # The model architecture we used: in our case PPO
#                env_id=env_id, # Name of the environment
#                eval_env=eval_env, # Evaluation Environment
#                repo_id=repo_id, # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
#                commit_message=commit_message)

## 7. Replay the Video with the Agent

In [10]:
# Replay the video with the agent
from IPython.display import HTML
from base64 import b64encode

video_file = '/content/drive/MyDrive/Courses/02-ML-Advance-Hillel/Lesson-19/replay-ppo-1m-ep-v1.mp4'

# Encode video to base64 to embed in HTML
video = open(video_file, "rb").read()
video_encoded = b64encode(video).decode()

# Display the video
HTML(f'''
<video width="640" height="480" controls>
  <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
  Your browser does not support the video tag.
</video>
''')


# Play the video with the agent from file in internet
# %%html
# <video controls autoplay><source src="https://" type="video/mp4"></video>

[Link](https://huggingface.co/vagi/ppo-LunarLander-v2) to my repo on HuggingFace